In [1]:
import pandas as pd

TEST = '/kaggle/input/house-prices-advanced-regression-techniques/test.csv'
TRAIN = '/kaggle/input/house-prices-advanced-regression-techniques/train.csv'

test_raw_df = pd.read_csv(filepath_or_buffer=TEST, )
submission_df = test_raw_df[['Id']]
test_raw_df = test_raw_df.drop(columns=['Id'])
train_raw_df = pd.read_csv(filepath_or_buffer=TRAIN, ).drop(columns=['Id'])
train_raw_df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


Here's our data processing pipeline; let's do as much of it as we can before we start comparing models.

In [2]:
from sklearn.impute import KNNImputer


TARGET = 'SalePrice'
keys = [key for key, value in train_raw_df.drop(columns=[TARGET]).dtypes.to_dict().items() if str(value) in {'int64', 'float64'}]
COLUMNS = [key for key in keys if test_raw_df[key].isna().sum() == 0]

# our imputing is a work in progress, and we think at the moment that we benefit from 
# using KNN imputing on these three columns is best
knn_columns = [key for key in keys if 0 < test_raw_df[key].isna().sum() < 5]
knn_columns = COLUMNS + ['BsmtFinSF1',  'BsmtUnfSF', 'TotalBsmtSF']

imputer = KNNImputer()
train_knn_df = pd.DataFrame(data=imputer.fit_transform(X=train_raw_df[knn_columns]), columns=knn_columns)
test_knn_df = pd.DataFrame(data=imputer.transform(X=test_raw_df[knn_columns]), columns=knn_columns)

# now get the dummies we want to bolt on
other_columns = [column for column in test_raw_df.columns if column not in COLUMNS and column not in knn_columns and 0 == train_raw_df[column].isna().sum() and 0 == test_raw_df[column].isna().sum()]
dummy_columns = other_columns[:3]

train_dummies_df = pd.get_dummies(data=train_raw_df[dummy_columns])
test_dummies_df = pd.get_dummies(data=test_raw_df[dummy_columns])

train_df = pd.concat(axis='columns', objs=[train_knn_df, train_dummies_df,])
train_df[TARGET] = train_raw_df[TARGET].tolist()
test_df = pd.concat(axis='columns', objs=[test_knn_df, test_dummies_df])
print(train_df.shape, test_df.shape)

(1460, 39) (1459, 38)


The competition uses RMSE, so we need an RMSE metric.

In [3]:
import numpy as np
from sklearn.metrics import mean_squared_error

def rmse(y_true, y_pred) -> float:
    return mean_squared_error(squared=False, y_true=np.log(1 + y_true), y_pred=np.log(1 + y_pred))

In [4]:
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor


ACTIVATION = 'identity'
CRITERION = 'absolute_error'
MODELS = {
    'ridge +': Ridge(tol=1e-4, random_state=2024, max_iter=10000, positive=True, solver='lbfgs'),
    'neural network' : MLPRegressor(hidden_layer_sizes=(400, 200, 100,), activation=ACTIVATION, solver='adam', alpha=1e-3, batch_size='auto', 
                      learning_rate='adaptive', learning_rate_init=1e-2, power_t=0.5, max_iter=1000, shuffle=True, random_state=2024,
                      tol=1e-5, verbose=False, warm_start=False, momentum=0.8, nesterovs_momentum=True, early_stopping=False, beta_1=0.9, 
                      beta_2=0.999, epsilon=1e-08, n_iter_no_change=20, max_fun=15000),
    'tree' : DecisionTreeRegressor(criterion=CRITERION, splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1,
                             min_weight_fraction_leaf=0.0, max_features=None, random_state=2024, max_leaf_nodes=None,
                             min_impurity_decrease=0.0, ccp_alpha=0.0, ),
}

for name, model in MODELS.items():
    scores = []
    for random_state in range(10):
        X_train, X_test, y_train, y_test = train_test_split(train_df.drop(columns=[TARGET]), train_df[TARGET], test_size=0.20, random_state=random_state)
        try:
            model.fit(X=X_train, y=y_train)
            score = rmse(y_true=y_test, y_pred=model.predict(X=X_test))
            scores.append(score)
        except ValueError:
            pass

    print('{:5.4f} : {}'.format(sum(scores)/len(scores), name))

0.2087 : ridge +


/tmp/ipykernel_18/71397580.py:5: RuntimeWarning: invalid value encountered in log
  return mean_squared_error(squared=False, y_true=np.log(1 + y_true), y_pred=np.log(1 + y_pred))
/tmp/ipykernel_18/71397580.py:5: RuntimeWarning: invalid value encountered in log
  return mean_squared_error(squared=False, y_true=np.log(1 + y_true), y_pred=np.log(1 + y_pred))
/tmp/ipykernel_18/71397580.py:5: RuntimeWarning: invalid value encountered in log
  return mean_squared_error(squared=False, y_true=np.log(1 + y_true), y_pred=np.log(1 + y_pred))


0.2409 : neural network
0.2040 : tree


Let's spend some time tuning our ridge regression parameters:
* alpha : l2 regularization strength

Remember that we have learned elsewhere that our error metric is sensitive to the train/test split, so to fairly compare different alpha values we need to take the mean RMSE across a range of random state values.


In [5]:
from plotly import express

alpha_df = pd.DataFrame()
best_score = 0.9
for ialpha in range(40):
    model = Ridge(alpha=(1 + ialpha) / 2.0, tol=1e-4, random_state=2024, max_iter=10000, positive=True, solver='lbfgs')
    scores = []
    for random_state in range(10):
        X_train, X_test, y_train, y_test = train_test_split(train_df.drop(columns=[TARGET]), train_df[TARGET], test_size=0.20, random_state=random_state)
        model.fit(X=X_train, y=y_train)
        scores.append(rmse(y_true=y_test, y_pred=model.predict(X=X_test)))
    alpha_df[ialpha] = scores
    mean_score = sum(scores)/len(scores)
    if mean_score < best_score:
        best_score = mean_score
        MODELS['best'] = model
        

express.line(data_frame=alpha_df.mean().to_frame().reset_index(), y=0)

In [6]:
SUBMISSION = '/kaggle/working/submission.csv'

name = 'best'
submission_df[TARGET] = MODELS[name].fit(X=train_df.drop(columns=[TARGET]), y=train_df[TARGET]).predict(X=test_df)
submission_df.to_csv(path_or_buf=SUBMISSION, index=False)
print('done.')

done.
